In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pathlib
data_dir = pathlib.Path("/kaggle/input/dermnetdataset/DermnetDataset/train")

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
import PIL
import matplotlib.pyplot as plt
from pathlib import Path

# Defining the directories and number of images to display
data_dir = Path("/content/drive/MyDrive/Dermnetdataset/train")
dirs = ["Acne and Rosacea Photos", "Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions", "Atopic Dermatitis Photos","Eczema Photos","Hair Loss Photos Alopecia and other Hair Diseases","Melanoma Skin Cancer Nevi and Moles","Psoriasis pictures Lichen Planus and related diseases","Seborrheic Keratoses and other Benign Tumors","Tinea Ringworm Candidiasis and other Fungal Infections","Urticaria Hives"]
num_images = 3

# Creating a figure with subplots
fig, axes = plt.subplots(len(dirs), num_images, figsize=(15, 30))

# Loop over the directories and images to display
for i, d in enumerate(dirs):
    image_paths = list((data_dir / d).glob("*"))
    for j in range(num_images):
        # Opening the image using PIL and display it on the subplot
        img = PIL.Image.open(str(image_paths[j]))
        axes[i, j].imshow(img)
        axes[i, j].axis("off")
        # Adding a title to the subplot with the directory name
        if j == 0:
            axes[i, j].set_title(d)

# Adjusting the spacing and layout of the subplots
plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout()
plt.show()


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
     image_size=(256,256),
  subset="training",
  seed=123)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
    image_size=(256,256),
  subset="validation",
  seed=123)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
    ])

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(256, 256, 3)),
  layers.BatchNormalization(),

  layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.l2(0.01), activation='relu'),
  layers.MaxPooling2D((2,2)),

  layers.Dropout(0.5),

  layers.Conv2D(64, 3, padding='same', kernel_regularizer=regularizers.l2(0.01), activation='relu'),
  layers.MaxPooling2D((2,2)),

  layers.Conv2D(64, 3, padding='same', kernel_regularizer=regularizers.l2(0.01), activation='relu'),
  layers.MaxPooling2D((2,2)),

  layers.Conv2D(64, 3, padding='same', kernel_regularizer=regularizers.l2(0.01), activation='relu'),
  layers.MaxPooling2D((2,2)),

  layers.Dropout(0.5),

  layers.Flatten(),
  layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
  layers.BatchNormalization(),
  layers.Dense(num_classes)
])
model.build(input_shape=input_shape)

In [ ]:

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
custom_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    mode='min'
)

In [ ]:
epochs=22
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  verbose=1,
  callbacks=[custom_early_stopping]
)

In [ ]:
model.save("skin.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/test/Eczema Photos/03DermatitisArm1.jpg"

img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.metrics import Metric
from tensorflow.python.keras.utils import losses_utils

@tf.keras.utils.register_keras_serializable()
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(tf.round(y_pred), tf.float32)

        true_positives = tf.reduce_sum(y_true * y_pred)
        false_positives = tf.reduce_sum((1 - y_true) * y_pred)
        false_negatives = tf.reduce_sum(y_true * (1 - y_pred))

        self.true_positives.assign_add(true_positives)
        self.false_positives.assign_add(false_positives)
        self.false_negatives.assign_add(false_negatives)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + losses_utils.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + losses_utils.epsilon())

        f1_score = 2 * (precision * recall) / (precision + recall + losses_utils.epsilon())
        return f1_score

    def get_config(self):
        base_config = super(F1Score, self).get_config()
        return {**base_config}

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[F1Score()])


In [ ]:
model.save("skin_model.h5")

In [ ]:
model = keras.models.load_model("/content/skin.h5")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array


model = tf.keras.models.load_model('/content/skin.h5')

image_path = '/content/drive/MyDrive/Dermnetdataset/train/Eczema Photos/Dyshidrosis-20.jpg'
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
input_data = image_array.reshape(1, 256, 256, 3)

predictions = model.predict(input_data)
print(predictions)
score = tf.nn.softmax(predictions[0])
print(score)

In [ ]:
print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Seborrheic Keratoses and other Benign Tumors/accessory-trachus-4.jpg"

img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Tinea Ringworm Candidiasis and other Fungal Infections/13TineaGroin143.jpg"

img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Acne and Rosacea Photos/07Rhinophymaq.jpg"

img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/actinic-cheilitis-sq-cell-lip-145.jpg"

img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Hair Loss Photos Alopecia and other Hair Diseases/acne-keloidalis-11.jpg"
img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Psoriasis pictures Lichen Planus and related diseases/08LichenPlanusTongue.jpg"
img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Tinea Ringworm Candidiasis and other Fungal Infections/13Candida040701.jpg"
img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
fil = "/content/drive/MyDrive/Dermnetdataset/train/Urticaria Hives/PUPPP-12.jpg"
img = tf.keras.utils.load_img(
    fil, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {}."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# Convert to TensorFlow Lite (Optional)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
